# GeoScripting Recap 2022

<i>Author: Nikolaos Kolaxidis <br/>
Date: 26.07.2022</i>

## Exercise 1 + 3

In [ ]:
import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from osgeo import gdal
import time

print(os.getcwd())
os.chdir(r"F:\Geoscripting with Python\geoscripting\recap") #-- change
gdal.UseExceptions()

starttime1 = time.time()

# Handling the input file
# ----------
try:
    dem = gdal.Open(sys.argv[1])
except RuntimeError as err:
    print('Unable to open a file because of: ', str(err), '\nPlease check your input files.')
    sys.exit(1)

cols = dem.RasterXSize
rows = dem.RasterYSize
geotransform = dem.GetGeoTransform()
originX = geotransform[0]
originY = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]

# Read the input files as Numpy arrays and preprocess them
# ----------
band_dem = dem.GetRasterBand(1)
try:
    data_dem = band_dem.ReadAsArray(0, 0, cols, rows).astype(float)
except Exception as err:
    print("Error occured: " + str(err))
    sys.exit(1)
# Replace the Nodata value with np.nan
data_dem = np.where(data_dem == band_dem.GetNoDataValue(), np.nan, data_dem) # np.where(condition, on true, on false)

# Converting feet to meter
def feet2meters(data):
    """
    Converts feet to meters by using 0,3048 as factor
    param feet: input values as feet
    output: returns the converted float value in meters
    """
    meter = data * 0.3048
    return meter

dem_m = feet2meters(data_dem)

# Handling the output file
# ----------
outfile = sys.argv[2]
outRaster = None
try:
    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(outfile, cols, rows, 1, gdal.GDT_Float32)
except Exception as err:
    print("Error occured when creating outRaster: " + str(err))
    sys.exit(1)
# Georeference new Raster based on the dsm
outRaster.SetGeoTransform(geotransform)
# Select raster band to write into
outband = outRaster.GetRasterBand(1)
# Write to outRaster with values from outband
try:
    outband.WriteArray(dem_m)
    outband.FlushCache()
except Exception as err:
    print("Error occured when writing into the outRaster: " + str(err))
    sys.exit(1)
outRaster = None

# Print whether the conversion worked
print ("Finished converting the raster %s from feet to meters. \n Result stored in %s " % (sys.argv[1], outfile))

# Determine the highest elevation and its coordinates
high_pt = np.nanmax(dem_m)
coords = np.where(dem_m == high_pt)
try:
    x = originX + 0.5 * pixelWidth + pixelWidth * coords[1]
    y = originY + 0.5 * pixelHeight + pixelHeight * coords[0]
except Exception as err:
    print("Error occured when transforming the coordinates: " + str(err))
    sys.exit(1)

stoptime1 = time.time()

# Print out the information to user
print("The highest point of the dataset is %.2f at the location %.2fE %.2fN. \n The script took %.2f seconds to run." % (high_pt, x, y, stoptime1-starttime1))

# Plot the outRaster
plt.figure(figsize=(20,20))
plt.imshow(dem_m, cmap='terrain')
plt.title("DEM of Heidelberg in meters")
plt.xlabel('X-Axis')
plt.ylabel('Y-Axis')
# Colorbar should have the same height as the plot
im_ratio = dem_m.shape[0]/dem_m.shape[1]
plt.colorbar(fraction=0.046*im_ratio)
plt.plot(coords[1], coords[0], marker='^', ms=10, color='black', label='Highest elevation: %.2f m' % (high_pt))
plt.legend()
plt.show()
plt.close()

## Exercise 2

In [ ]:
import os, sys
from osgeo import ogr
import time

# print(os.getcwd())
os.chdir(r"F:\Geoscripting with Python\geoscripting\recap") #-- change

starttime2 = time.time()

driver = ogr.GetDriverByName('ESRI Shapefile')
try:
    vect_data = driver.Open(sys.argv[1], 0) # read
except Exception as err:
    print("Error occured when loading the vector file: " + str(err))
    sys.exit(1)
layer = vect_data.GetLayer()
layer.SetAttributeFilter("ALIASNAME = 'HEIDELBERG'")
try:
    feature = layer.GetNextFeature()
    output = feature['Annual_mea']
except:
    print("\nERROR: Specified column might not be available or cannot be accessed.")

vect_data.Destroy()

stoptime2 = time.time()

# Print out the information to user
print("The annual mean temperature of %s is %s °C. \n The script took %.2f seconds to run." % (sys.argv[2], output, stoptime2-starttime2))